In [ ]:
# for running in collab, sagemaker etc.
import sys
import os


def _append_sys_path_of_utils():
    """
    Appends an utils directory to a system path so it is visible for the script.
    """
    UTILS_RELATIVE_PATH = "../../utils/"

    absolute_path_of_utils = os.path.abspath(UTILS_RELATIVE_PATH)
    if absolute_path_of_utils:
        if absolute_path_of_utils in sys.path:
            print(f"Absolute path already in sys.path: {absolute_path_of_utils}")
        else:
            print(f"Appended absolute path: {absolute_path_of_utils}")
            sys.path.insert(0, absolute_path_of_utils)
    else:
        print("Absolute path of utils module not found.")


_append_sys_path_of_utils()

In [ ]:
import wandb

In [ ]:
!wandb login

In [ ]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model, decomposition, manifold, preprocessing
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve, roc_curve, auc, accuracy_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
from tensorflow.keras import layers
import pickle
import time
from numpy import load
from matplotlib import pyplot
import pickle
import argparse

import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

from tensorflow.compat.v1.keras.layers import Input, Dense, LeakyReLU, Conv2D, MaxPooling2D, UpSampling2D,  Concatenate
from tensorflow.compat.v1.keras.models import Model
from tensorflow.compat.v1.keras.layers import Dense, Reshape, Flatten
from tensorflow.compat.v1.keras.layers import Dropout,BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import mse, binary_crossentropy, logcosh
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"

from scipy.stats import wasserstein_distance
import pandas as pd
from utils import sum_channels_parallel as sum_channels_parallel
from sklearn.metrics import mean_absolute_error
from datetime import datetime

In [ ]:
!pip install -U tf2onnx

In [ ]:
!pip install protobuf==3.20.1

In [ ]:
!pwd

In [ ]:
# It can be used to reconstruct the model identically.
model = keras.models.load_model("../../experiments/sdi-gan_15_2133_08_05_2023_21_20/models/best_model/gen_sdi_gan_62.h5")
tf.saved_model.save(model, "gen_sdi_gan_62")

In [ ]:
!python3 -m tf2onnx.convert --saved-model gen_sdi_gan_62 --output "gen_sdi_gan_62_epoch.onnx"

In [ ]:
SAVE_EXPERIMENT_DATA = True

### Load and process data

In [ ]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time

from IPython import display
import sklearn
from sklearn.preprocessing import StandardScaler
from datetime import datetime

In [ ]:
data = pd.read_pickle('../../data/data_proton_20_2312_neutron_20_3360_padded.pkl')
print('Loaded: ',  data.shape, "max:", data.max())

data_cond = pd.read_pickle('../../data/data_cond_stddev_photonsum_20_2312_20_3360_padded.pkl')
print('Loaded cond: ',  data_cond.shape, "max:", data_cond.values.max(), "min:", data_cond.values.min())

In [ ]:
# calculate min max proton, neutron sum
photon_sum_proton_min, photon_sum_proton_max = data_cond.proton_photon_sum.min(), data_cond.proton_photon_sum.max()
photon_sum_neutron_min, photon_sum_neutron_max = data_cond.neutron_photon_sum.min(), data_cond.neutron_photon_sum.max()

In [ ]:
data_cond.drop(columns=['proton_photon_sum', 'neutron_photon_sum', 'group_number_proton', 'group_number_neutron'], inplace=True)
data_cond.columns, len(data_cond.columns)

In [ ]:
data_cond.head()

In [ ]:
STRENGTH = 0.1

DATE_STR = datetime.now().strftime("%d_%m_%Y_%H_%M")

NAME = "sdi-gan-padded"

wandb_run_name = f"{int(photon_sum_proton_min)}_{int(photon_sum_proton_max)}_{DATE_STR}"

EXPERIMENT_DIR_NAME = f"experiments/{NAME}_{int(photon_sum_proton_min)}_{int(photon_sum_proton_max)}_{DATE_STR}"

print("Experiment DIR: ", EXPERIMENT_DIR_NAME)

In [ ]:
def create_dir(path):
    if SAVE_EXPERIMENT_DATA:
        isExist = os.path.exists(path)
        if not isExist:
            os.makedirs(path)

In [ ]:
def save_scales(model_name, scaler_means, scaler_scales):
    out_fnm = f"{model_name}_scales.txt"
    res = "#means"
    for mean_ in scaler_means:
        res += "\n" + str(mean_)
    res += "\n\n#scales"
    for scale_ in scaler_scales:
        res += "\n" + str(scale_)

    if SAVE_EXPERIMENT_DATA:
        filepath = f"../../{EXPERIMENT_DIR_NAME}/scales/"
        create_dir(filepath)
        with open(filepath+out_fnm, mode="w") as f:
            f.write(res)

In [ ]:
data_cond["cond"] = data_cond["Energy"].astype(str) +"|"+ data_cond["Vx"].astype(str) +"|"+  data_cond["Vy"].astype(str) +"|"+ data_cond["Vz"].astype(str) +"|"+  data_cond["Px"].astype(str) +"|"+  data_cond["Py"].astype(str) +"|"+ data_cond["Pz"].astype(str) +"|"+  data_cond["mass"].astype(str) +"|"+  data_cond["charge"].astype(str)
data_cond.head()

In [ ]:
data_cond_id = data_cond[["cond"]].reset_index()
data_cond_id

In [ ]:
# select a random index of the same conditional data
# shuffle the data and merge it according to the conditional data. Pick the first index of the grouped conditional data
# if some unique conditional data has only one index of sample, then pair it with the same index 
ids = data_cond_id.merge(data_cond_id.sample(frac=1), on=["cond"], how="inner").groupby("index_x").first()
ids = ids["index_y"]
ids

### Idea
Eliminate samples that are unique in terms of conditional data leaving only these responses that have few samples grouped by condtional data

Now we will have two conditional datasets and two response datasets

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data_cond

In [ ]:
data = np.log(data+1)
data = np.float32(data)
print("data max", data.max(), "min", data.min())

data_2 = data[ids]

data_cond = data_cond.drop(columns="cond")

scaler_proton = MinMaxScaler()
std_proton = data_cond["std_proton"].values.reshape(-1,1)
std_proton = np.float32(std_proton)
std_proton = scaler_proton.fit_transform(std_proton)
print("std max", std_proton.max(), "min", std_proton.min())

scaler_neutron = MinMaxScaler()
std_neutron = data_cond["std_neutron"].values.reshape(-1,1)
std_neutron = np.float32(std_neutron)
std_neutron = scaler_neutron.fit_transform(std_neutron)
print("std max", std_neutron.max(), "min", std_neutron.min())

In [ ]:
scaler = StandardScaler()
data_cond = np.float32(data_cond.drop(columns=["std_proton", "std_neutron"]))
data_cond = scaler.fit_transform(data_cond)
print("cond max", data_cond.max(), "min", data_cond.min())

In [ ]:
data_2.shape

In [ ]:
x_train, x_test, x_train_2, x_test_2, y_train, y_test, std_proton_train, std_proton_test, std_neutron_train, std_neutron_test = train_test_split(data, data_2, data_cond, std_proton, std_neutron, test_size=0.2, shuffle=False)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape, std_proton_train.shape, std_proton_test.shape, std_neutron_train.shape, std_neutron_test.shape)

In [ ]:
#save scales
if SAVE_EXPERIMENT_DATA:
    save_scales("Proton", scaler.mean_, scaler.scale_)

In [ ]:
BATCH_SIZE = 128

# Training dataset

# datasets that in each index contain two samples from the same conditional data
dataset = tf.data.Dataset.from_tensor_slices(x_train).batch(batch_size=BATCH_SIZE)
dataset_2 = tf.data.Dataset.from_tensor_slices(x_train_2).batch(batch_size=BATCH_SIZE)
# conditional data
dataset_cond = tf.data.Dataset.from_tensor_slices(y_train).batch(batch_size=BATCH_SIZE)
# standard deviation for each conditional data samples
dataset_std_proton = tf.data.Dataset.from_tensor_slices(std_proton_train).batch(batch_size=BATCH_SIZE)
dataset_std_neutron = tf.data.Dataset.from_tensor_slices(std_neutron_train).batch(batch_size=BATCH_SIZE)
# shuffled conditional data
fake_cond =  tf.data.Dataset.from_tensor_slices(y_train).shuffle(12800).batch(batch_size=BATCH_SIZE)

# zipped data
dataset_with_cond = tf.data.Dataset.zip((dataset, dataset_2, dataset_cond, dataset_std_proton, dataset_std_neutron, fake_cond)).shuffle(12800)

# Validation dataset

val_dataset = tf.data.Dataset.from_tensor_slices(x_test).batch(batch_size=BATCH_SIZE)
val_dataset_2 = tf.data.Dataset.from_tensor_slices(x_test_2).batch(batch_size=BATCH_SIZE)
val_dataset_cond = tf.data.Dataset.from_tensor_slices(y_test).batch(batch_size=BATCH_SIZE)
val_dataset_std_proton = tf.data.Dataset.from_tensor_slices(std_proton_test).batch(batch_size=BATCH_SIZE)
val_dataset_std_neutron = tf.data.Dataset.from_tensor_slices(std_neutron_test).batch(batch_size=BATCH_SIZE)
val_fake_cond =  tf.data.Dataset.from_tensor_slices(y_test).shuffle(12800).batch(batch_size=BATCH_SIZE)
val_dataset_with_cond = tf.data.Dataset.zip((val_dataset, val_dataset_2, val_dataset_cond, val_dataset_std_proton, val_dataset_std_neutron, val_fake_cond)).shuffle(12800)

### Define Model

In [ ]:
from tensorflow.compat.v1.keras.layers import Input, Dense, LeakyReLU, Conv2D, MaxPooling2D, UpSampling2D,  Concatenate
from tensorflow.compat.v1.keras.models import Model
from tensorflow.compat.v1.keras.layers import Dense, Reshape, Flatten
from tensorflow.compat.v1.keras.layers import Dropout,BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import mse, binary_crossentropy, logcosh
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K

from tensorflow import keras

In [ ]:
# OLD Architecture with latent dimension 16 failed, achieved mode collapse

In [ ]:
# OLD Architecture

latent_dim = 8
cond_dim = 9

############################ generator ############################

x = Input(shape=(latent_dim,))
cond = Input(shape=(cond_dim,))
inputs = Concatenate(axis=1)([x, cond])

layer_1 = Dense(128*2)(inputs)
layer_1_bd = Dropout(0.2)(BatchNormalization()(layer_1))
layer_1_a = LeakyReLU(alpha=0.1)(layer_1_bd)

layer_2 = Dense(128*20*12)(layer_1_a)
layer_2_bd = Dropout(0.2)(BatchNormalization()(layer_2))
layer_2_a = LeakyReLU(alpha=0.1)(layer_2_bd)

reshaped = Reshape((20,12,128))(layer_2_a)
reshaped_s = UpSampling2D(size=(3,2))(reshaped)

conv1 = Conv2D(256, kernel_size=(2, 2), padding='valid')(reshaped_s)
conv1_bd = Dropout(0.2)(BatchNormalization()(conv1))
conv1_a = LeakyReLU(alpha=0.1)(conv1_bd)
conv1_a_s = UpSampling2D(size=(1,2))(conv1_a)

conv2 = Conv2D(128, kernel_size=2)(conv1_a_s)
conv2_bd = Dropout(0.2)(BatchNormalization()(conv2))
conv2_a = LeakyReLU(alpha=0.1)(conv2_bd)

conv3 = Conv2D(64, kernel_size=2)(conv2_a)
conv3_bd = Dropout(0.2)(BatchNormalization()(conv3))
conv3_a = LeakyReLU(alpha=0.1)(conv3_bd)

outputs = Conv2D(2, kernel_size=(2, 1), activation='relu')(conv3_a)

generator = Model([x, cond], outputs, name='generator')
generator.summary()

############################ discriminator ############################

input_img = Input(shape=[56,44,2],name='input_img')
conv1 = Conv2D(32, kernel_size=3)(input_img)
conv1_bd = Dropout(0.2)(BatchNormalization()(conv1))
conv1_a = LeakyReLU(alpha=0.1)(conv1_bd)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1_a)

conv2 = Conv2D(16, kernel_size=3)(pool1)
conv2_bd = Dropout(0.2)(BatchNormalization()(conv2))
conv2_a = LeakyReLU(alpha=0.1)(conv2_bd)
pool2 = MaxPooling2D(pool_size=(2, 1))(conv2_a)

flat = Flatten()(pool2)
cond = Input(shape=(cond_dim,))
inputs2 = Concatenate(axis=1)([flat, cond])
layer_1 = Dense(128)(inputs2)
layer_1_bd = Dropout(0.2)(BatchNormalization()(layer_1))
layer_1_a = LeakyReLU(alpha=0.1)(layer_1_bd)

layer_2 = Dense(64)(layer_1_a)
layer_2_bd = Dropout(0.2)(BatchNormalization()(layer_2))
layer_2_a = LeakyReLU(alpha=0.1)(layer_2_bd)
outputs = Dense(1, activation='sigmoid')(layer_2_a)

discriminator = Model([input_img, cond], [outputs, layer_2_a], name='discriminator')
discriminator.summary()

### Idea
Increase number of latent dimensions with the old architecture

In [ ]:
# # architecture 4

# latent_dim = 20
# cond_dim = 9

# ############################ generator ############################

# x = Input(shape=(latent_dim,))
# cond = Input(shape=(cond_dim,))
# inputs = Concatenate(axis=1)([x, cond])

# layer_1 = Dense(256*2)(inputs)
# layer_1_bd = Dropout(0.2)(BatchNormalization()(layer_1))
# layer_1_a = LeakyReLU(alpha=0.1)(layer_1_bd)

# layer_2 = Dense(256*16*12)(layer_1_a)
# layer_2_bd = Dropout(0.2)(BatchNormalization()(layer_2))
# layer_2_a = LeakyReLU(alpha=0.1)(layer_2_bd)

# reshaped = Reshape((16,12,256))(layer_2_a)
# reshaped_s = UpSampling2D(size=(4, 2))(reshaped)

# conv1 = Conv2D(256, kernel_size=(3, 2), padding='valid')(reshaped_s)
# conv1_bd = Dropout(0.2)(BatchNormalization()(conv1))
# conv1_a = LeakyReLU(alpha=0.1)(conv1_bd)
# conv1_a_s = UpSampling2D(size=(1, 2))(conv1_a)

# conv2 = Conv2D(128, kernel_size=(4, 2))(conv1_a_s)
# conv2_bd = Dropout(0.2)(BatchNormalization()(conv2))
# conv2_a = LeakyReLU(alpha=0.1)(conv2_bd)

# conv3 = Conv2D(64, kernel_size=(1, 2))(conv2_a)
# conv3_bd = Dropout(0.2)(BatchNormalization()(conv3))
# conv3_a = LeakyReLU(alpha=0.1)(conv3_bd)

# outputs = Conv2D(2, kernel_size=(4, 1), activation='relu')(conv3_a)

# generator = Model([x, cond], outputs, name='generator')
# generator.summary()

# ############################ discriminator ############################

# input_img = Input(shape=[56, 44, 2], name='input_img')
# conv1 = Conv2D(64, kernel_size=3)(input_img)
# conv1_bd = Dropout(0.2)(BatchNormalization()(conv1))
# conv1_a = LeakyReLU(alpha=0.1)(conv1_bd)
# pool1 = MaxPooling2D(pool_size=(2, 2))(conv1_a)

# conv2 = Conv2D(32, kernel_size=3)(pool1)
# conv2_bd = Dropout(0.2)(BatchNormalization()(conv2))
# conv2_a = LeakyReLU(alpha=0.1)(conv2_bd)
# pool2 = MaxPooling2D(pool_size=(2, 1))(conv2_a)

# flat = Flatten()(pool2)
# cond = Input(shape=(cond_dim,))

# inputs2 = Concatenate(axis=1)([flat, cond])
# layer_1 = Dense(128)(inputs2)
# layer_1_bd = Dropout(0.2)(BatchNormalization()(layer_1))
# layer_1_a = LeakyReLU(alpha=0.1)(layer_1_bd)

# layer_2 = Dense(64)(layer_1_a)
# layer_2_bd = Dropout(0.2)(BatchNormalization()(layer_2))
# layer_2_a = LeakyReLU(alpha=0.1)(layer_2_bd)

# outputs = Dense(2, activation='sigmoid')(layer_2_a)

# discriminator = Model([input_img, cond], [outputs, layer_2_a], name='discriminator')
# discriminator.summary()

In [ ]:
# # architecture 3

# latent_dim = 16
# cond_dim = 9

# ############################ generator ############################

# x = Input(shape=(latent_dim,))
# cond = Input(shape=(cond_dim,))
# inputs = Concatenate(axis=1)([x, cond])

# layer_1 = Dense(128*2)(inputs)
# layer_1_bd = Dropout(0.2)(BatchNormalization()(layer_1))
# layer_1_a = LeakyReLU(alpha=0.1)(layer_1_bd)

# layer_2 = Dense(128*16*12)(layer_1_a)
# layer_2_bd = Dropout(0.2)(BatchNormalization()(layer_2))
# layer_2_a = LeakyReLU(alpha=0.1)(layer_2_bd)

# reshaped = Reshape((16,12,128))(layer_2_a)
# reshaped_s = UpSampling2D(size=(4, 2))(reshaped)

# conv1 = Conv2D(256, kernel_size=(3, 2), padding='valid')(reshaped_s)
# conv1_bd = Dropout(0.2)(BatchNormalization()(conv1))
# conv1_a = LeakyReLU(alpha=0.1)(conv1_bd)
# conv1_a_s = UpSampling2D(size=(1, 2))(conv1_a)

# conv2 = Conv2D(128, kernel_size=(4, 2))(conv1_a_s)
# conv2_bd = Dropout(0.2)(BatchNormalization()(conv2))
# conv2_a = LeakyReLU(alpha=0.1)(conv2_bd)

# conv3 = Conv2D(64, kernel_size=(1, 2))(conv2_a)
# conv3_bd = Dropout(0.2)(BatchNormalization()(conv3))
# conv3_a = LeakyReLU(alpha=0.1)(conv3_bd)

# outputs = Conv2D(2, kernel_size=(4, 1), activation='relu')(conv3_a)

# generator = Model([x, cond], outputs, name='generator')
# generator.summary()

# ############################ discriminator ############################

# input_img = Input(shape=[56, 44, 2], name='input_img')
# conv1 = Conv2D(64, kernel_size=3)(input_img)
# conv1_bd = Dropout(0.2)(BatchNormalization()(conv1))
# conv1_a = LeakyReLU(alpha=0.1)(conv1_bd)
# pool1 = MaxPooling2D(pool_size=(2, 2))(conv1_a)

# conv2 = Conv2D(32, kernel_size=3)(pool1)
# conv2_bd = Dropout(0.2)(BatchNormalization()(conv2))
# conv2_a = LeakyReLU(alpha=0.1)(conv2_bd)
# pool2 = MaxPooling2D(pool_size=(2, 1))(conv2_a)

# flat = Flatten()(pool2)
# cond = Input(shape=(cond_dim,))

# inputs2 = Concatenate(axis=1)([flat, cond])
# layer_1 = Dense(128)(inputs2)
# layer_1_bd = Dropout(0.2)(BatchNormalization()(layer_1))
# layer_1_a = LeakyReLU(alpha=0.1)(layer_1_bd)

# layer_2 = Dense(64)(layer_1_a)
# layer_2_bd = Dropout(0.2)(BatchNormalization()(layer_2))
# layer_2_a = LeakyReLU(alpha=0.1)(layer_2_bd)

# outputs = Dense(2, activation='sigmoid')(layer_2_a)

# discriminator = Model([input_img, cond], [outputs, layer_2_a], name='discriminator')
# discriminator.summary()

In [ ]:
# Second bigger architecture
# latent_dim = 20
# cond_dim = 9

# ############################ generator ############################

# x = Input(shape=(latent_dim,))
# cond = Input(shape=(cond_dim,))
# inputs = Concatenate(axis=1)([x, cond])

# layer_1 = Dense(256*2)(inputs)
# layer_1_bd = Dropout(0.2)(BatchNormalization()(layer_1))
# layer_1_a = LeakyReLU(alpha=0.1)(layer_1_bd)

# layer_2 = Dense(256*16*12)(layer_1_a)
# layer_2_bd = Dropout(0.2)(BatchNormalization()(layer_2))
# layer_2_a = LeakyReLU(alpha=0.1)(layer_2_bd)

# reshaped = Reshape((16,12,256))(layer_2_a)
# reshaped_s = UpSampling2D(size=(4, 2))(reshaped)

# conv1 = Conv2D(512, kernel_size=(3, 2), padding='valid')(reshaped_s)
# conv1_bd = Dropout(0.2)(BatchNormalization()(conv1))
# conv1_a = LeakyReLU(alpha=0.1)(conv1_bd)
# conv1_a_s = UpSampling2D(size=(1, 2))(conv1_a)

# conv2 = Conv2D(256, kernel_size=(4, 2))(conv1_a_s)
# conv2_bd = Dropout(0.2)(BatchNormalization()(conv2))
# conv2_a = LeakyReLU(alpha=0.1)(conv2_bd)

# conv3 = Conv2D(128, kernel_size=(1, 2))(conv2_a)
# conv3_bd = Dropout(0.2)(BatchNormalization()(conv3))
# conv3_a = LeakyReLU(alpha=0.1)(conv3_bd)

# outputs = Conv2D(2, kernel_size=(4, 1), activation='relu')(conv3_a)

# generator = Model([x, cond], outputs, name='generator')
# generator.summary()

# ############################ discriminator ############################

# input_img = Input(shape=[56, 44, 2],name='input_img')
# conv1 = Conv2D(64, kernel_size=3)(input_img)
# conv1_bd = Dropout(0.2)(BatchNormalization()(conv1))
# conv1_a = LeakyReLU(alpha=0.1)(conv1_bd)
# pool1 = MaxPooling2D(pool_size=(2, 2))(conv1_a)

# conv2 = Conv2D(128, kernel_size=3)(pool1)
# conv2_bd = Dropout(0.2)(BatchNormalization()(conv2))
# conv2_a = LeakyReLU(alpha=0.1)(conv2_bd)
# pool2 = MaxPooling2D(pool_size=(2, 1))(conv2_a)

# flat = Flatten()(pool2)
# cond = Input(shape=(cond_dim,))

# inputs2 = Concatenate(axis=1)([flat, cond])
# layer_1 = Dense(64)(inputs2)
# layer_1_bd = Dropout(0.2)(BatchNormalization()(layer_1))
# layer_1_a = LeakyReLU(alpha=0.1)(layer_1_bd)

# layer_2 = Dense(32)(layer_1_a)
# layer_2_bd = Dropout(0.2)(BatchNormalization()(layer_2))
# layer_2_a = LeakyReLU(alpha=0.1)(layer_2_bd)

# outputs = Dense(2, activation='sigmoid')(layer_2_a)

# discriminator = Model([input_img, cond], [outputs, layer_2_a], name='discriminator')
# discriminator.summary()

In [ ]:
# # OLD architecture
# latent_dim = 20
# cond_dim = 10

# ############################ generator ############################

# x = Input(shape=(latent_dim,))
# cond = Input(shape=(cond_dim,))
# inputs = Concatenate(axis=1)([x, cond])

# layer_1 = Dense(128*2)(inputs)
# layer_1_bd = Dropout(0.2)(BatchNormalization()(layer_1))
# layer_1_a = LeakyReLU(alpha=0.1)(layer_1_bd)

# layer_2 = Dense(128*28*22)(layer_1_a)
# layer_2_bd = Dropout(0.2)(BatchNormalization()(layer_2))
# layer_2_a = LeakyReLU(alpha=0.1)(layer_2_bd)

# reshaped = Reshape((28, 22, 128))(layer_2_a)
# # reshaped_s = UpSampling2D()(reshaped)

# conv1 = Conv2D(128, kernel_size=1)(reshaped)
# conv1_bd = Dropout(0.2)(BatchNormalization()(conv1))
# conv1_a = LeakyReLU(alpha=0.1)(conv1_bd)
# conv1_a_s = UpSampling2D()(conv1_a)

# conv2 = Conv2D(64, kernel_size=1)(conv1_a_s)
# conv2_bd = Dropout(0.2)(BatchNormalization()(conv2))
# conv2_a = LeakyReLU(alpha=0.1)(conv2_bd)

# outputs = Conv2D(1, kernel_size=1,activation='relu')(conv2_a)

# generator = Model([x, cond], outputs, name='generator')
# generator.summary()

# ############################ discriminator ############################

# input_img = Input(shape=[56,44,2],name='input_img')
# conv1 = Conv2D(32, kernel_size=3)(input_img)
# conv1_bd = Dropout(0.2)(BatchNormalization()(conv1))
# conv1_a = LeakyReLU(alpha=0.1)(conv1_bd)
# pool1 = MaxPooling2D(pool_size=(2, 2))(conv1_a)
# conv2 = Conv2D(16, kernel_size=3)(pool1)
# conv2_bd = Dropout(0.2)(BatchNormalization()(conv2))
# conv2_a = LeakyReLU(alpha=0.1)(conv2_bd)
# pool2 = MaxPooling2D(pool_size=(2, 2))(conv2_a)
# flat = Flatten()(pool2)
# cond = Input(shape=(cond_dim,))

# inputs2 = Concatenate(axis=1)([flat, cond])
# layer_1 = Dense(128)(inputs2)
# layer_1_bd = Dropout(0.2)(BatchNormalization()(layer_1))
# layer_1_a = LeakyReLU(alpha=0.1)(layer_1_bd)
# layer_2 = Dense(64)(layer_1_a)
# layer_2_bd = Dropout(0.2)(BatchNormalization()(layer_2))
# layer_2_a = LeakyReLU(alpha=0.1)(layer_2_bd)
# outputs = Dense(1, activation='sigmoid')(layer_2_a)

# discriminator = Model([input_img, cond], outputs, name='discriminator')
# discriminator.summary()

### IDEA
Why do we only take into account a loss of only single generated image

In [ ]:
# def discriminator_loss(real_output, fake_output, fake_output_2):
#     real_loss = cross_entropy(tf.ones_like(real_output), real_output)
#     fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
#     fake_loss_2 = cross_entropy(tf.zeros_like(fake_output_2), fake_output_2)
#     total_loss = real_loss + fake_loss + fake_loss_2
    
#     # update state of accuracy of real and false images
#     d_acc_r.update_state(tf.ones_like(real_output), real_output)
#     d_acc_f.update_state(tf.zeros_like(fake_output), fake_output)
#     return total_loss

In [ ]:
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    
    # update state of accuracy of real and false images
    d_acc_r.update_state(tf.ones_like(real_output), real_output)
    d_acc_f.update_state(tf.zeros_like(fake_output), fake_output)
    return total_loss

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [ ]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)
d_acc_r = keras.metrics.BinaryAccuracy(name="d_acc_r", threshold=0.5)
d_acc_f = keras.metrics.BinaryAccuracy(name="d_acc_r", threshold=0.5)
g_acc = keras.metrics.BinaryAccuracy(name="g_acc_g", threshold=0.5)

### IDEA
Why do we only take into accoutn a single genrated image, not from two latent codes

In [ ]:
# def generator_loss(step, fake_output, fake_output_2,
#                    fake_latent, fake_latent_2,
#                    noise, noise_2,
#                    std_proton, std_neutron):

#     g_acc.update_state(tf.ones_like(fake_output), fake_output)
#     g_acc.update_state(tf.ones_like(fake_output_2), fake_output_2)

#     crossentropy_loss = cross_entropy(tf.ones_like(fake_output), fake_output) + cross_entropy(tf.ones_like(fake_output_2), fake_output_2)

#     div = tf.math.divide(tf.reduce_mean(tf.abs(fake_latent - fake_latent_2),(1)), tf.reduce_mean(tf.abs(noise-noise_2),(1)))

#     div_loss_proton = std_proton * STRENGTH / (div + 1e-5)
#     div_loss_neutron = std_neutron * STRENGTH / (div + 1e-5)

#     div_loss_proton = tf.reduce_mean(tf.math.multiply(tf.reduce_mean(std_proton,(1)), div_loss_proton))
#     div_loss_neutron = tf.reduce_mean(tf.math.multiply(tf.reduce_mean(std_neutron,(1)), div_loss_neutron))
#     div_loss = div_loss_proton + div_loss_neutron
#     return crossentropy_loss + div_loss, div_loss

### IDEA
Should we take mean of diversity loss or just sum the up?

In [ ]:
def generator_loss(step, fake_output,
                   fake_latent, fake_latent_2,
                   noise, noise_2,
                   std_proton, std_neutron):

    g_acc.update_state(tf.ones_like(fake_output), fake_output)

    crossentropy_loss = cross_entropy(tf.ones_like(fake_output), fake_output)

    div = tf.math.divide(tf.reduce_mean(tf.abs(fake_latent - fake_latent_2),(1)), tf.reduce_mean(tf.abs(noise-noise_2),(1)))

    div_loss_proton = std_proton * STRENGTH / (div + 1e-5)
    div_loss_neutron = std_neutron * STRENGTH / (div + 1e-5)

    div_loss_proton = tf.reduce_mean(tf.math.multiply(tf.reduce_mean(std_proton,(1)), div_loss_proton))
    div_loss_neutron = tf.reduce_mean(tf.math.multiply(tf.reduce_mean(std_neutron,(1)), div_loss_neutron))

    # average diversity loss
    div_loss = div_loss_proton + div_loss_neutron
    return crossentropy_loss + div_loss, div_loss

In [ ]:
EPOCHS = 200
noise_dim = latent_dim
num_examples_to_generate = 16

START_GENERATING_IMG_FROM_IDX = 20
# Seed to reuse for generating samples for comparison during training
seed = tf.random.normal([num_examples_to_generate, noise_dim])
seed_cond = y_test[START_GENERATING_IMG_FROM_IDX:START_GENERATING_IMG_FROM_IDX+num_examples_to_generate]

In [ ]:
wandb.finish()
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Generative Models for CERN Fast Simulations",
    name=wandb_run_name,
    # track hyperparameters and run metadata
    config={
    "Model": NAME,
    "dataset": "proton_neutron_data",
    "epochs": EPOCHS,
    "Date": DATE_STR,
    "latent_dimension": latent_dim,
    "Proton_min": photon_sum_proton_min,
    "Proton_max": photon_sum_proton_max,
    "Experiment_dir_name": EXPERIMENT_DIR_NAME,
    "batch_size": BATCH_SIZE
    },
    tags=[f"proton_min_{photon_sum_proton_min}",
          f"proton_max_{photon_sum_proton_max}",
          f"neutron_min_{photon_sum_neutron_min}",
          f"neutron_max_{photon_sum_neutron_max}",
          f"gan_strength_{STRENGTH}", "sdi-gan"]
)

In [ ]:
from scipy.stats import wasserstein_distance
import pandas as pd
from utils import sum_channels_parallel
from sklearn.metrics import mean_absolute_error

org=np.exp(x_test)-1
ch_org = np.array(org).reshape(-1,56,44)
ch_org = pd.DataFrame(sum_channels_parallel(ch_org)).values
del org


def calculate_ws_ch(n_calc):
    ws= [0,0,0,0,0]
    for j in range(n_calc):
        z = np.random.normal(0,1,(x_test.shape[0],latent_dim))
        z_c = y_test
        results = generator.predict([z,z_c])
        results = np.exp(results)-1
        try:
            ch_gen = np.array(results).reshape(-1,56,44)
            ch_gen = pd.DataFrame(sum_channels_parallel(ch_gen)).values
            for i in range(5):
                ws[i] = ws[i] + wasserstein_distance(ch_org[:,i], ch_gen[:,i])
            ws =np.array(ws)
        except ValueError as e:
            print(e)

    ws = ws/n_calc
    ws_mean = ws.sum()/5
    print("ws mean",f'{ws_mean:.2f}', end=" ")
    for n, score in enumerate(ws):
        print("ch"+str(n+1),f'{score:.2f}',end=" ")
    return ws_mean


In [37]:
# def generator_loss(step, fake_output, fake_output_2,
#                    fake_latent, fake_latent_2,
#                    noise, noise_2,
#                    std_proton, std_neutron):

#     g_acc.update_state(tf.ones_like(fake_output), fake_output)
#     g_acc.update_state(tf.ones_like(fake_output_2), fake_output_2)

#     crossentropy_loss = cross_entropy(tf.ones_like(fake_output), fake_output) + cross_entropy(tf.ones_like(fake_output_2), fake_output_2)

#     div = tf.math.divide(tf.reduce_mean(tf.abs(fake_latent - fake_latent_2),(1)), tf.reduce_mean(tf.abs(noise-noise_2),(1)))

#     div_loss_proton = std_proton * STRENGTH / (div + 1e-5)
#     div_loss_neutron = std_neutron * STRENGTH / (div + 1e-5)

#     div_loss_proton = tf.reduce_mean(tf.math.multiply(tf.reduce_mean(std_proton,(1)), div_loss_proton))
#     div_loss_neutron = tf.reduce_mean(tf.math.multiply(tf.reduce_mean(std_neutron,(1)), div_loss_neutron))
#     div_loss = div_loss_proton + div_loss_neutron
#     return crossentropy_loss + div_loss, div_loss

### IDEA
Should we take mean of diversity loss or just sum the up?

In [38]:
def generator_loss(step, fake_output,
                   fake_latent, fake_latent_2,
                   noise, noise_2,
                   std_proton, std_neutron):

    g_acc.update_state(tf.ones_like(fake_output), fake_output)

    crossentropy_loss = cross_entropy(tf.ones_like(fake_output), fake_output)

    div = tf.math.divide(tf.reduce_mean(tf.abs(fake_latent - fake_latent_2),(1)), tf.reduce_mean(tf.abs(noise-noise_2),(1)))

    div_loss_proton = std_proton * STRENGTH / (div + 1e-5)
    div_loss_neutron = std_neutron * STRENGTH / (div + 1e-5)

    div_loss_proton = tf.reduce_mean(tf.math.multiply(tf.reduce_mean(std_proton,(1)), div_loss_proton))
    div_loss_neutron = tf.reduce_mean(tf.math.multiply(tf.reduce_mean(std_neutron,(1)), div_loss_neutron))

    # average diversity loss
    div_loss = div_loss_proton + div_loss_neutron
    return crossentropy_loss + div_loss, div_loss

In [39]:
EPOCHS = 200
noise_dim = latent_dim
num_examples_to_generate = 16

START_GENERATING_IMG_FROM_IDX = 20
# Seed to reuse for generating samples for comparison during training
seed = tf.random.normal([num_examples_to_generate, noise_dim])
seed_cond = y_test[START_GENERATING_IMG_FROM_IDX:START_GENERATING_IMG_FROM_IDX+num_examples_to_generate]

In [40]:
wandb.finish()
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Generative Models for CERN Fast Simulations",
    name=wandb_run_name,
    # track hyperparameters and run metadata
    config={
    "Model": NAME,
    "dataset": "proton_neutron_data",
    "epochs": EPOCHS,
    "Date": DATE_STR,
    "latent_dimension": latent_dim,
    "Proton_min": photon_sum_proton_min,
    "Proton_max": photon_sum_proton_max,
    "Experiment_dir_name": EXPERIMENT_DIR_NAME,
    "batch_size": BATCH_SIZE
    },
    tags=[f"proton_min_{photon_sum_proton_min}",
          f"proton_max_{photon_sum_proton_max}",
          f"neutron_min_{photon_sum_neutron_min}",
          f"neutron_max_{photon_sum_neutron_max}",
          f"gan_strength_{STRENGTH}", "sdi-gan"]
)

wandb: Currently logged in as: bedkowski-patrick (nlp-wut-2023). Use `wandb login --relogin` to force relogin


In [41]:
from scipy.stats import wasserstein_distance
import pandas as pd
from utils import sum_channels_parallel
from sklearn.metrics import mean_absolute_error

org=np.exp(x_test)-1
ch_org = np.array(org).reshape(-1,56,44)
ch_org = pd.DataFrame(sum_channels_parallel(ch_org)).values
del org


def calculate_ws_ch(n_calc):
    ws= [0,0,0,0,0]
    for j in range(n_calc):
        z = np.random.normal(0,1,(x_test.shape[0],latent_dim))
        z_c = y_test
        results = generator.predict([z,z_c])
        results = np.exp(results)-1
        try:
            ch_gen = np.array(results).reshape(-1,56,44)
            ch_gen = pd.DataFrame(sum_channels_parallel(ch_gen)).values
            for i in range(5):
                ws[i] = ws[i] + wasserstein_distance(ch_org[:,i], ch_gen[:,i])
            ws =np.array(ws)
        except ValueError as e:
            print(e)

    ws = ws/n_calc
    ws_mean = ws.sum()/5
    print("ws mean",f'{ws_mean:.2f}', end=" ")
    for n, score in enumerate(ws):
        print("ch"+str(n+1),f'{score:.2f}',end=" ")
    return ws_mean


In [42]:
@tf.function
def train_step(batch, step):
    # dataset, dataset_2, dataset_cond, dataset_std_proton, dataset_std_neutron, fake_cond
    images, images_2, cond, std_proton, std_neutron, noise_cond = batch
    step=step
    BATCH_SIZE = tf.shape(images)[0]
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    noise_2 = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # for the same conditional data generate two images from different noises
        generated_images = generator([noise, noise_cond], training=True)
        generated_images_2 = generator([noise_2, noise_cond], training=True)
        
        # produce if real image is real or fake
        real_output, real_latent = discriminator([images, cond], training=True)
        # real_output_2,real_latent_2  = discriminator([images_2,cond], training=True)
        
        # produce if generated images from two different latent codes are real or fake
        fake_output, fake_latent = discriminator([generated_images, noise_cond], training=True)
        fake_output_2, fake_latent_2 = discriminator([generated_images_2, noise_cond], training=True)

        gen_loss, div_loss = generator_loss(step, fake_output,
                                            fake_latent, fake_latent_2,
                                            noise, noise_2,
                                            std_proton, std_neutron)
        disc_loss = discriminator_loss(real_output, fake_output)


    #         generated_images = generator([noise,noise_cond], training=True)

    #         real_output = discriminator([images,cond], training=True)
    #         fake_output = discriminator([generated_images, noise_cond], training=True)

    #         gen_loss = generator_loss(step, fake_output)
    #         real_loss, fake_loss = discriminator_loss(real_output, fake_output)
    #         disc_loss = real_loss + fake_loss

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss, div_loss


In [43]:
# If model achieves WS metric less or equal to this number, its weights will be saved
WS_MEAN_SAVE_THRESHOLD = 20


if SAVE_EXPERIMENT_DATA:
    filepath_mod = f"../../{EXPERIMENT_DIR_NAME}/models/"
    create_dir(filepath_mod)

history = []
def train(dataset, epochs):
    experiment_start = time.time()
    tf_step = tf.Variable(0, dtype=float)
    step=0

    # generate first image
    generate_and_save_images(generator,
                             epochs,
                             [seed, seed_cond])

    for epoch in range(epochs):
        start = time.time()

        gen_loss_epoch = []
        div_loss_epoch = []
        disc_loss_epoch = []
        for batch in dataset:
            gen_loss, disc_loss, div_loss =train_step(batch,tf_step)

            history.append([gen_loss,disc_loss,
                100*d_acc_r.result().numpy(),
                100*d_acc_f.result().numpy(),
                100*g_acc.result().numpy(),
                ])
            tf_step.assign_add(1)
            step = step+1

            gen_loss_epoch.append(gen_loss)
            disc_loss_epoch.append(disc_loss)
            div_loss_epoch.append(div_loss)
            if step % 100 == 0:
                print("%d [D real acc: %.2f%%] [D fake acc: %.2f%%] [G acc: %.2f%%] "% (
                    step,
                    100*d_acc_r.result().numpy(),
                    100*d_acc_f.result().numpy(),
                    100*g_acc.result().numpy()))

        plot = generate_and_save_images(generator,
                                 epoch,
                                 [seed, seed_cond])

        ws_mean = calculate_ws_ch(min(epoch//5+1,5))

        if SAVE_EXPERIMENT_DATA:
            if ws_mean <= WS_MEAN_SAVE_THRESHOLD:
                # Save the model every epoch
                generator.compile()
                # discriminator.compile()
                generator.save((os.path.join(filepath_mod, "gen_"+NAME + "_"+ str(epoch) +".h5")))
                # discriminator.save((os.path.join(filepath_mod, "disc_"+NAME + "_"+ str(epoch) +".h5")))
                # np.savez(os.path.join(filepath_mod, "history_"+NAME+".npz"),np.array(history))

        wandb.log({
            'ws_mean': ws_mean,
            'gen_loss': np.mean(gen_loss_epoch),
            'div_loss': np.mean(div_loss_epoch),
            'disc_loss': np.mean(disc_loss_epoch),
            'epoch': epoch,
            'plot': wandb.Image(plot),
            'experiment_time': time.time()-experiment_start
        })

        plt.close('all')

        print('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    return history

In [44]:
if SAVE_EXPERIMENT_DATA:
    filepath_img = f"../../{EXPERIMENT_DIR_NAME}/images/"
    create_dir(filepath_img)

    
def generate_and_save_images(model, epoch, test_input):
    START_INDEX = 6
    SUPTITLE_TXT = f"\nModel: GAN proton data" \
               f"\nPhotonsum interval: [{photon_sum_proton_min}, {photon_sum_proton_max}]" \
               f"\nEPOCH: {epoch}"

    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)
    """
    predictions has shape (n_samples, 56, 44, 2). First channel has proton data, second has neutrons
    """
    fig = plt.figure(figsize=(15,4))

    plt.title(f"EPOCH {epoch}")

    subfigs = fig.subfigures(1, 4)

    for particle_num, subfig in enumerate(subfigs.flat):  # iterate over 4 particles
        subfig.suptitle(f'Particle {particle_num} response')
        axs = subfig.subplots(2, 2)
        for i, ax in enumerate(axs.flat):  # iterate over 4 images of single particle
            m_2 = i % 2  # 0 if proton, 1 if neutron
            if i < 2:
                # Real response
                x = x_test[START_INDEX+particle_num][:,:,m_2].reshape(56,44)
                axs[i//2, m_2].set_title("neutron" if m_2 else "proton")
            else:
                # Generated response
                x = predictions[START_INDEX+particle_num].numpy()[:,:,m_2].reshape(56,44)
            axs[i//2, m_2].set_axis_off()
            im = axs[i//2, m_2].imshow(x, interpolation='none', cmap='gnuplot')
            fig.colorbar(im, ax=axs[i//2, m_2])

    if SAVE_EXPERIMENT_DATA:
        plt.savefig(os.path.join(filepath_img, 'image_at_epoch_{:04d}.png'.format(epoch)))
    
    return fig

### Train model

In [ ]:
history = train(dataset_with_cond, EPOCHS)

100 [D real acc: 28.84%] [D fake acc: 79.39%] [G acc: 20.61%] 
200 [D real acc: 41.35%] [D fake acc: 75.96%] [G acc: 24.04%] 
206/206 [==============================] - 4s 17ms/step
ws mean 85345.42 ch1 208.35 ch2 213.12 ch3 218.16 ch4 424678.04 ch5 1409.44 Time for epoch 1 is 147.5119824409485 sec
300 [D real acc: 50.50%] [D fake acc: 71.60%] [G acc: 28.40%] 
400 [D real acc: 55.58%] [D fake acc: 69.57%] [G acc: 30.43%] 
206/206 [==============================] - 3s 16ms/step
ws mean 759654153374984593997824.00 ch1 15862.25 ch2 3781326286550367037554688.00 ch3 170691.98 ch4 16944480324555928240128.00 ch5 72158681.55 Time for epoch 2 is 131.0457751750946 sec
500 [D real acc: 58.91%] [D fake acc: 68.29%] [G acc: 31.71%] 
600 [D real acc: 61.11%] [D fake acc: 68.38%] [G acc: 31.62%] 
206/206 [==============================] - 3s 16ms/step
ws mean 1384009587.86 ch1 5380.35 ch2 1973242082.92 ch3 444736.87 ch4 4760731.37 ch5 4941595007.80 Time for epoch 3 is 132.08433890342712 sec
700 [D re

In [ ]:
import seaborn as sns

sns.set_context('notebook', font_scale = 1)

In [ ]:
# total_loss, reconstruction_loss, kl_loss
history_losses = np.array([[float(loss) for loss in losses] for losses in history])

In [ ]:
generator_loss = history_losses[:,0]
disc_loss = history_losses[:,1]
discriminator_r_acc = history_losses[:,2]
discriminator_f_acc = history_losses[:,3]
generator_acc = history_losses[:,4]

In [ ]:
a4_dims = (14, 5)
def print_loss(loss_values, loss_str):
    fig, ax = plt.subplots(figsize=a4_dims)
    sns.lineplot(loss_values)
    plt.xscale('log')
    ax.set_title(f"{loss_str} loss in each epoch")
    ax.set_xlabel("Epoch number")
    ax.set_ylabel(f"{loss_str} loss value")
    # fig.save(f"../images/{EXPERIMENT_NAME}/{loss_str}_loss")

In [ ]:
print_loss(generator_loss, "Generator")
print_loss(disc_loss, "Discriminator")
print_loss(discriminator_r_acc, "Discriminator r accuracy")
print_loss(discriminator_f_acc, "Discriminator f accuracy")
print_loss(generator_acc, "Generator accuracy")